In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import os

In [2]:
re_set_name = 'Legendary-Duelists-Season-3'

In [28]:
def is_there_another_page(set_name, i):
    #turn this into a function where database and length of database are the outputs.

    cardmarket_result = requests.get('https://www.cardmarket.com/en/YuGiOh/Products/Singles/{}?sortBy=collectorsnumber_asc&site='.format(re_set_name) + str(i))
    src = cardmarket_result.content
    soup = BeautifulSoup(src, 'html.parser')
    card_price_info = soup.find_all(attrs = {'class':'col'})

    lo_names = []
    lo_numbers = []
    lo_rarities = []
    lo_links = []

    names = []
    numbers = []
    rarities = []
    links = []

    for entry in card_price_info:
        if entry.find('a') != None:
            cardmarket_url = entry.find('a')
            if (cardmarket_url.attrs['href'] != 'https://www.facebook.com/CardmarketYugioh') and (cardmarket_url.attrs['href'] != '/en/YuGiOh/Policies/GeneralTermsAndConditions'):
                links.append('https://www.cardmarket.com/' + cardmarket_url.attrs['href'])


        if entry.find(attrs = {'class' : 'col-10 col-md-8 px-2 flex-column align-items-start justify-content-center'}) != None:
            name = entry.find(attrs = {'class' : 'col-10 col-md-8 px-2 flex-column align-items-start justify-content-center'})
            if name.text != 'Name':
                re_name = re.sub('\"','', name.text)
                re_name = re.sub(':','', re_name)
                re_name = re.sub('/','', re_name)
                re_name = re.sub('\?','', re_name)
                names.append(re_name)

        if entry.find(attrs = {'class' : 'col-md-2 d-none d-lg-flex has-content-centered'}) != None:
            number = entry.find(attrs = {'class' : 'col-md-2 d-none d-lg-flex has-content-centered'})
            if number.text != 'Number':
                numbers.append(number.text)


        if entry.find(attrs = {'class' : 'icon'}) != None:
            if 'title' in list(entry.find(attrs = {'class' : 'icon'}).attrs.keys()):
                rarity = entry.find(attrs = {'class' : 'icon'}).attrs['title']
                if rarity != 'Facebook - Cardmarket':
                    rarities.append(rarity)
                    
    uk_links = []
    for link in links:
        link = link + '?sellerCountry=13'
        uk_links.append(link)

    page_set_content = pd.DataFrame({'Card Name': names, 'Card Number' : numbers, 'Rarity': rarities, 'Cardmarket Links': links, 'Cardmarket UK links': uk_links})

    return (page_set_content, len(page_set_content))

In [29]:
def cardmarket_df(set_name):
    
    i = 1
    dfs = []
    
    while is_there_another_page(set_name, i)[1] == 20:
        
        dl_pair = is_there_another_page(set_name, i)
        dfs.append(dl_pair[0])
        
        if len(dfs) > 1:
            if dfs[-1].equals(dfs[-2]):
                return pd.concat(dfs[:-1], ignore_index = True)
        i = i+1
        
    else:
        dfs.append(is_there_another_page(set_name, i)[0])

    return pd.concat(dfs, ignore_index = True)

In [27]:
df = cardmarket_df(re_set_name)

In [30]:
print(len(df))

198


In [31]:
df.head(40)

,Card Name,Card Number,Rarity,Cardmarket Links,Cardmarket UK links
0,Sangan (V.1 - Ultra Rare),001,Ultra Rare,https://www.cardmarket.com//en/YuGiOh/Products...,https://www.cardmarket.com//en/YuGiOh/Products...
1,Sangan (V.2 - Ultra Rare),001,Ultra Rare,https://www.cardmarket.com//en/YuGiOh/Products...,https://www.cardmarket.com//en/YuGiOh/Products...
2,Sangan (V.3 - Ultra Rare),001,Ultra Rare,https://www.cardmarket.com//en/YuGiOh/Products...,https://www.cardmarket.com//en/YuGiOh/Products...
3,Dark Necrofear (V.3 - Ultra Rare),002,Ultra Rare,https://www.cardmarket.com//en/YuGiOh/Products...,https://www.cardmarket.com//en/YuGiOh/Products...
4,Dark Necrofear (V.2 - Ultra Rare),002,Ultra Rare,https://www.cardmarket.com//en/YuGiOh/Products...,https://www.cardmarket.com//en/YuGiOh/Products...
5,Dark Necrofear (V.1 - Ultra Rare),002,Ultra Rare,https://www.cardmarket.com//en/YuGiOh/Products...,https://www.cardmarket.com//en/YuGiOh/Products...
6,Jowgen the Spiritualist,003,Common,https://www.cardmarket.com//en/YuGiOh/Products...,https://www.cardmarket.com//en/YuGiOh/Products...
7,Gernia (V.1 - Ultra Rare),004,Ultra Rare,https://www.cardmarket.com//en/YuGiOh/Products...,https://www.cardmarket.com//en/YuGiOh/Products...
8,Gernia (V.2 - Ultra Rare),004,Ultra Rare,https://www.cardmarket.com//en/YuGiOh/Products...,https://www.cardmarket.com//en/YuGiOh/Products...
9,Gernia (V.3 - Ultra Rare),004,Ultra Rare,https://www.cardmarket.com//en/YuGiOh/Products...,https://www.cardmarket.com//en/YuGiOh/Products...


In [ ]:
#we have a solution!!